#**0**. Mounting Google **Drive**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

#1. Importing Dataset & Mini-Analysis

**Importing `numpy`**



---





In [0]:
import numpy as np

**Loading Dataset**

Kindly change the path if required!

In [0]:
X = np.load('/content/drive/My Drive/colab/ML AI Dataset/X_train.npy')
Y = np.load('/content/drive/My Drive/colab/ML AI Dataset/Y_train.npy')

## Mini Analysis

Find the maximum value in **X** and normalize it. 

*(For images, it is sufficient to divide by 255, which is done later)*

---



In [0]:
np.max(X[...,0])

255.0

**Checking thw distribution**

It should be around 0.5 to make sure that we have good distribution of exammples, i.e, good distribution of 1's and 0's

In [0]:
np.mean(Y)

0.5016826923076924

Numbers of examples in the data...

In [0]:
M = X.shape[0]
M

4160

Verifying the width/height of image...

In [0]:
X.shape[1]

50

## Splitting dataset we have into training, validation and test set

We will train on the Training set, validate it (to have an eye on overfitting/underfitting) using validation set and make a prediction using the test set made to check how well our model is performing.

In [0]:
shuffle = np.random.permutation(M)

I have used a split of 85%-10%-5% (Training-Validation-Test) 

In [0]:
X_train = X[:int(0.85*M),...]
Y_train = Y[:int(0.85*M),...]
Y_train = Y_train.reshape(Y_train.shape[0],1)

X_val = X[int(0.85*M):int(0.95*M),...]
Y_val = Y[int(0.85*M):int(0.95*M),...]
Y_val = Y_val.reshape(Y_val.shape[0],1)

X_test = X[int(0.95*M):,...]
Y_test = Y[int(0.95*M):,...]
Y_test = Y_test.reshape(Y_test.shape[0],1)

Making sure that we have a good distribution of training set

In [0]:
np.mean(Y_train)


0.5016968325791855

The shape of training set obtained...

In [0]:
X_train.shape

(3536, 50, 50, 3)

**Normalise the features!**

In [0]:
X_train /= 255
X_val /= 255
X_test /= 255

# 2. Building the Model

**Importing `keras` with Tensorflow backend**

**Note:** The model is NOT Sequential

In [0]:
from keras.models import Model
from keras.layers import Input,BatchNormalization, Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Flatten, Dropout, Dense, Concatenate, Add, UpSampling2D, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator

**Data Augmentation**

We have got very less training examples. So one way to reduce overfitting is to use Data Augmentation (so that our model can be feeded with some more variations of the training data, like flipping, rotating, zooming, etc. the original training data).

In [0]:
aug = ImageDataGenerator(
		rotation_range=12.5,
		zoom_range=0.13,
		width_shift_range=0.12,
		height_shift_range=0.12,
		horizontal_flip=True,
    vertical_flip = True,
		fill_mode="nearest")


**Model**

I have used a simple Model, whose schema is given below:


---
---
---

**[INPUT]**

---

**[Convolution]** *size=(3,3) filters=32 stride=None padding=Same*

**[Batch Normalization]** *Along last axis*

**[Activation]** *function=relu*

**[Dropout]** *factor=0.5*

**[Convolution]** *size=(3,3) filters=32 stride=None padding=Valid*

**[Batch Normalization]** *Along last axis*

**[Activation]** *function=relu*

**[Dropout]** *factor=0.5*

**[Max Pooling]** *size=(2,2)*   --- (A)

---


**[Convolution]** *size=(3,3) filters=64 stride=None padding=Same*

**[Batch Normalization]** *Along last axis*

**[Activation]** *function=relu*

**[Dropout]** *factor=0.3*


**[(Seperable) Convolution]** *size=(3,3) filters=64 stride=None padding=Same*

**[Batch Normalization]** *Along last axis*

**[Activation]** *function=leaky_relu* --(X1)

**[Convolution]** *size=(3,3) filters=64 stride=None padding=Same*

**[Batch Normalization]** *Along last axis*  --(Y1)

**[Add]** (X1+Y1)

**[Activation]** *function=leaky_relu*

**[Dropout]** *factor=0.3*

**[Max Pooling]** *size=(2,2)*   --- (B)

---

**[Convolution]** *size=(3,3) filters=128 stride=None padding=Same*

**[Batch Normalization]** *Along last axis*

**[Activation]** *function=relu*



**[(Seperable) Convolution]** *size=(3,3) filters=128 stride=None padding=Same*

**[Batch Normalization]** *Along last axis*

**[Activation]** *function=leaky_relu* --(X1)

**[Convolution]** *size=(3,3) filters=128 stride=None padding=Same*

**[Batch Normalization]** *Along last axis*  --(Y1)

**[Add]** (X1+Y1)

**[Activation]** *function=leaky_relu*

**[Dropout]** *factor=0.25*

**[Max Pooling]** *size=(2,2)*   --- (C)

---
---

**[Up Sampling]**(A) *size=(2,2)*  --- (A')

**[Up Sampling]**(B) *size=(4,4)*  --- (B')

**[Up Sampling]**(C) *size=(8,8)*  --- (C')

---

**[Concatenate]** (A',B',C')

---

**[Global Average Pooling2D]**

**[Dense (Fully connected)]** *num_units=128,activation='relu'*

**[Dropout]** *factor=0.45*

**[Dense (Fully connected)]** *num_units=32,activation='relu'*

**[Dropout]** *factor=0.35*

**[Dense (Fully connected)]** *num_units=1,activation='sigmoid'*

---

**[OUTPUT]**

---
---
---




**Problems at Hand**


*   Going deeper can increase overfitting
*   Increasing number of hidden units can increase overfitting
*   Dropout is used to reduce overfitting and the values were tuned while training the model to get good results
*   Regularization was an option but didn't increase the results significantly. So it was dropped
 



***References***


*  Inspiration for 'adding' layers (Residual Layer) was from ResNet paper titled 
[Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) *by Kaiming He et al.*,2015

*The layers were tuned based on the results with time.*


In [0]:
inp = Input(shape=X_train.shape[1:])

x = Conv2D(filters=32, kernel_size=(3, 3), activation='linear', padding="same")(inp)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Conv2D(filters=32, kernel_size=(3, 3), activation='linear', padding="valid")(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x1 = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=(3, 3), activation='linear', padding="same")(x1)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)

y = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='linear', padding="same")(x)
y = BatchNormalization(axis=-1)(y)
y = LeakyReLU()(y)

x = Conv2D(filters=64, kernel_size=(3, 3), activation='linear', padding="same")(y)
x = BatchNormalization(axis=-1)(x)
x = Add()([x,y])

x = LeakyReLU()(x)
x = Dropout(0.3)(x)

x2 = MaxPooling2D(pool_size=(2, 2))(x)


x = Conv2D(filters=128, kernel_size=(3, 3), activation='linear', padding="same")(x2)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)

y = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='linear', padding="same")(x)
y = BatchNormalization(axis=-1)(y)
y = LeakyReLU()(y)

x = Conv2D(filters=128, kernel_size=(3, 3), activation='linear', padding="same")(y)
x = BatchNormalization(axis=-1)(x)
x = Add()([x,y])

x = LeakyReLU()(x)
x = Dropout(0.25)(x)

x3 = MaxPooling2D(pool_size=(2, 2))(x)


x1 = UpSampling2D(size=(2, 2))(x1)
x2 = UpSampling2D(size=(4, 4))(x2)
x3 = UpSampling2D(size=(8, 8))(x3)

x = Concatenate()([x1,x2,x3])
# x = Concatenate()([x1,x2])

x= GlobalAveragePooling2D()(x)
# x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.45)(x)
x = Dense(32,activation='relu')(x)
x = Dropout(0.35)(x)
x = Dense(1,activation='sigmoid')(x)


model = Model(inputs=inp,outputs=x)



In [0]:
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])
history = model.fit_generator(
	aug.flow(X_train, Y_train, batch_size=16),
	validation_data=(X_val, Y_val),
	epochs=128)

Epoch 1/128
221/221 [==============================] - 9s 42ms/step - loss: 0.6202 - accuracy: 0.7048 - val_loss: 0.6373 - val_accuracy: 0.7163
Epoch 2/128
221/221 [==============================] - 7s 33ms/step - loss: 0.5694 - accuracy: 0.7325 - val_loss: 0.6513 - val_accuracy: 0.5337
Epoch 3/128
221/221 [==============================] - 7s 32ms/step - loss: 0.5598 - accuracy: 0.7443 - val_loss: 0.6137 - val_accuracy: 0.6731
Epoch 4/128
221/221 [==============================] - 7s 33ms/step - loss: 0.5501 - accuracy: 0.7528 - val_loss: 0.7045 - val_accuracy: 0.5312
Epoch 5/128
221/221 [==============================] - 7s 33ms/step - loss: 0.5312 - accuracy: 0.7537 - val_loss: 0.6688 - val_accuracy: 0.5264
Epoch 6/128
221/221 [==============================] - 7s 32ms/step - loss: 0.5408 - accuracy: 0.7571 - val_loss: 0.6226 - val_accuracy: 0.6514
Epoch 7/128
221/221 [==============================] - 7s 32ms/step - loss: 0.5253 - accuracy: 0.7644 - val_loss: 0.6074 - val_accuracy:

In [0]:
# Without data augmentation
# model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])
# model.fit(X_train,Y_train,validation_data=(X_val,Y_val),epochs=64,batch_size=32)

The training is done... Let's run it on the test data to see how this model performs on unseen examples.

**Note:** Retraining the model might not give the same result due to randomness, like in Data Augmentation. The model was trained several times (iterated several times) to get this accuracy.

In [0]:
ans =model.predict(X_test)

Calculating the accuracy...

In [0]:
ans = ans.round()
np.mean((ans==Y_test).astype('int'))

0.7836538461538461

Let's calculate the Precision, Recall and F1 score using the `sklearn` inbuilt functions *(and this code was copied from internet)*

In [0]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

y_test = Y_test
y_pred = ans

print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))   

0.7538461538461538
0.7713133640552995
0.7624686542212316


Save the model for using it in the future...

**Note:** Change the path if required...

In [0]:
model.save('/content/drive/My Drive/colab/cancer.h5')

And the summary (info on paramters) of the model... *(didn't run it)*

In [0]:
model.summary()

# 3. Prediction

Loading the saved model..

In [1]:
from keras.models import load_model

Using TensorFlow backend.


In [0]:
model = load_model('/content/drive/My Drive/colab/cancer.h5')

Import the test data set **and normalise**

In [0]:
X_test = np.load('/content/drive/My Drive/colab/ML AI Dataset/X_test.npy')
X_test /= 255

In [28]:
X_test.shape

(1387, 50, 50, 3)

It's time to predict **and round the floating value to 0 or 1**!!

In [0]:
ans = model.predict(X_test)

In [0]:
ans = np.round(ans)

In [31]:
ans.mean()

0.5638068

**Save the prediction**

*  In `.npy` format

In [0]:
np.save('/content/drive/My Drive/colab/Y_out.npy',ans)

*  In `.csv` format

In [0]:
np.savetxt('/content/drive/My Drive/colab/subhalingam_d.csv', [ans[:,0]], delimiter='\n', fmt='%d' , header='Output')

Having a look at the prediction...

In [21]:
ans[:,0]

array([0., 0., 1., ..., 0., 0., 1.], dtype=float32)